[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/FlorianMarquardt/machine-learning-for-physicists/blob/master/2024/07_Tutorial_Q-learning_theory.ipynb)

# Reinforcement Learning Tutorial

Below we introduce reinforcement learning from the ground up. If you are already familiar with the concepts presented, feel free to skip the theory part and start directly with the exercises.

Reinforcement learning (RL) is a machine learning technique to approximately solve iterative decision problems. Prominent examples include game playing (e.g. Go: https://www.nature.com/articles/nature16961, Atari: https://arxiv.org/abs/1312.5602, Startcraft II: https://www.nature.com/articles/s41586-019-1724-z), robotic control (e.g. https://arxiv.org/abs/1808.00177), discovering alorithms (e.g. sorting algorithms now part of the standard C sorting library: https://www.nature.com/articles/s41586-023-06004-9), and fine-tuning of Large Language Models (including ChatGPT: https://arxiv.org/abs/2203.02155).

## Markov Decision Processes

RL can be applied to any problem that can be modeled as a Markov Decision Process (MDP). In an MDP, an agent interacts iteratively with an environment. At each time step $t$, the agent receives the current state of the environment $s_t$ (e.g., the state of a chessboard). At then chooses an action (e.g. moving a chess piece) by sampling from its policy $\pi(a_t|s_t)$, which is a probability distribution over all available actions given a state. The environment then samples a reward $r_t$ and its subsequent state $s_{t+1}$ from its transition probability distribution $p(s_{t+1}, r_t | s_t, a_t)$. This process is repeated during a so-called trajectory until the environment reaches a terminal state. The goal of the RL agent is to learn an ideal policy $\pi$ that maximizes the expected sum of rewards during a trajectory, i.e., the return
$$\mathbb{E}_\pi \left[\sum_{t=0}^{T-1} \gamma^t r_t\right],$$
where $T$ is the trajectory length, $\gamma \in (0, 1]$ is the discount factor, and the expectation value is taken over the agent's policy and the probabilistic environment. 

The discount factor $\gamma$ is often set to a value slightly less than $1$ to help train the agent by "not planning too far ahead". Note that this changes the agent's ideal policy in potentially undesirable ways. MDPs with finite trajectory length are called epsidoc and are considered in this tutorial. However, RL can also be applied to MDPs with infinite trajetory length by maximising the average reward or the discounted return with $\gamma <1$.

<center width="100%" style="padding:25px"><img src="MDP.001.jpeg" width="750px"></center>


There are two classes of RL algorithms: Policy methods and Q-learning. In this notebook, we discuss the theory of Q-learning

## Q-learning

The central object in Q-learning is the Q-function, which is defined as the expected value of the discounted sum of rewards after taking an action $a_t$ in a state $s_t$ and then following a policy $\pi$:
$$Q_\pi(s, a) = \mathbb{E}_\pi \left[\sum_{k=t}^{T-1} \gamma^{k} r_k \bigg| s_t = s, a_t = a\right].$$
For the Q-function of the ideal policy $Q^*(s, a)$ of the MDP we have (convince yourself that this is true!)
$$Q^*(s, a) = \mathbb{E}_\pi \left[r_{t+1} + \gamma \max_{a^\prime}Q^*(s_{t+1}, a^\prime) \bigg| s_t = s, a_t = a\right] = \sum_{s^\prime, r} p(s^\prime, r | s, a) \left[r + \gamma \max_{a^\prime}Q^*(s^\prime, a^\prime) \right].$$
This equation is called the Bellmann equation and is the foundation of all Q-learning based RL methods.

Vanilla Q-learning works by randomly initializing a Q-function and then interacting with the environment. Then the Q-function is iteratively updated according to
$$Q(s_t, a_t) \leftarrow Q(s_t, a_t) + \eta \left[r_t + \gamma \max_{a} Q(s_{t+1}, a) - Q(s_t, a_t) \right].$$
Note that according to the Bellman equation, the Q function of the ideal policy $Q^*$ is a fixed point of this update rule. It can be shown that as long as each state-action pair is visited infinitely often and the learning rate $\eta$ decays to zero, this update rule will converge to the Q-function of the optimal policy for arbitrarily initialized Q-functions (https://link.springer.com/article/10.1007/BF00992698). Once $Q^*$ is found, the optimal policy is simply to choose at each step the action that maximizes $Q^*$, i.e. $\argmax_a Q^*(s_t, a)$.
Typically, the policy chosen to interact with the environment during training is the so-called $\epsilon$ greedy policy: Here we choose a random action with probability $\epsilon$ and otherwise the action that maximizes the current Q-function, i.e. $\argmax_a Q(s_t, a)$. By setting $\epsilon > 0$, we ensure that all state-action pairs are in principle visited infinitely often.

### Deep Q-Learning
If the Q-function of the agent is not stored in a table but encoded by a neural network, the update rule changes to 
$$\theta \leftarrow \theta - \eta \nabla_\theta \left[r_t + \gamma \max_{a} Q_\theta(s_{t+1}, a) - Q_\theta(s_t, a_t) \right]^2.$$
Usually this gradient is averaged over a batch of experiences rather than a single step.
